# SECTION 1: Load Data, Train and Save Model

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import *
import matplotlib.pyplot as plt
import pickle
import numpy as np
import sklearn
import csv
tf.__version__

> Load data

In [ ]:
with open('Datasets/train_label.pkl', 'rb') as f:
    labels = pickle.load(f)
    
with open('Datasets/train_image.pkl', 'rb') as f:
    x_train_unshuffled = pickle.load(f)
    
with open('Datasets/test_image.pkl', 'rb') as f:
    x_test = pickle.load(f)


In [ ]:
# Reshaping Data. As required my the NN.
x_train_unshuffled = np.array(x_train_unshuffled).reshape(-1, 28, 28, 1)
x_test = np.array(x_test).reshape(-1, 28, 28, 1)

# Normalizing data. Mappind data from 0 to 1
x_train_unshuffled = tf.keras.utils.normalize(x_train_unshuffled,axis = 1 )
x_test = tf.keras.utils.normalize(x_test,axis = 1 )

> The labels in the dataset are: 0, 2, 3, 6. They are mapped with 4 classes of NN: 0, 1, 2, 3

In [ ]:
label_class_map = {
    0: 0,
    2: 1,
    3: 2,
    6: 3
}

class_label_map = {
    0: 0,
    1: 2,
    2: 3,
    3: 6
}

y_train_unshuffled = []

for i in range(len(labels)):
    y_train_unshuffled.append(label_class_map[labels[i]])
    
y_train_unshuffled = np.array(y_train_unshuffled)


> Shuffling data prior to training. I am using sklearn's shuffle utility for this. 

In [ ]:
x_train, y_train = sklearn.utils.shuffle(x_train_unshuffled, y_train_unshuffled)

> Basic NN Architecture | 2 Convolution Layers, 2 Dense Layers

> Training Accuracy: 99% | Validation Accuracy: 87%


In [ ]:
model = tf.keras.models.Sequential()

model.add(Conv2D(32, (3, 3),activation="relu", padding="same"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (2, 2), activation="relu", padding="same"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(128))
model.add(Activation('relu'))

#Output Layer: Number of neurons = number of classe1s.
model.add(Dense(4))
model.add(Activation('softmax'))

model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
              metrics = (['accuracy'])
             )
model.fit(x_train, y_train, epochs=50, shuffle = True, validation_split= 0)


In [ ]:
model.save('TrainedModels/midas_cnn_v2.model')

# SECTION 2: Load Saved Model, Run Predictions

In [ ]:
new_model = tf.keras.models.load_model('TrainedModels/midas_cnn_v2.model')
predictions = new_model.predict(x_test)
len(predictions)


> Testing some predictions

In [ ]:
index = 1600
plt.imshow(x_test[index].reshape(28,28))
class_label_map[np.argmax(predictions[index])]

In [ ]:
class_label_map[np.argmax(predictions[700])]

> Writing data to csv file

In [ ]:
with open('Mohd_Omama.csv', mode='w') as f:
    writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(['image_index', 'class'])
    
    for i in range(len(x_test)):
        print(class_label_map[np.argmax(predictions[i])])
        writer.writerow([i, class_label_map[np.argmax(predictions[i])] ])